## Speaker Diarization mit Pyannote

In diesem Workshop lernst du, wie man mithilfe der [pyannote.audio](https://github.com/pyannote/pyannote-audio) Bibliothek **Speaker Diarization** (Wer hat wann gesprochen?) durchführt.

Wir verwenden:
- **pyannote.audio** für das Pipeline-Modell

Voraussetzungen:
- Eine `.env`-Datei mit `PYANNOTE_HF_TOKEN` (dein Hugging Face Token) und `PYANNOTE_MODEL` (der Modellname, z. B. `"pyannote/speaker-diarization"` oder ähnliches).
- Ein Audiodatei im `.wav`-Format (oder ein kompatibles Format) zum Testen.

### 1. Imports

In [1]:
import os
import json
from dotenv import load_dotenv
from pyannote.audio import Pipeline


### 2. Laden von Umgebungsvariablen

- Wir greifen auf `PYANNOTE_HF_TOKEN` und `PYANNOTE_MODEL` zu, die in dieser Datei gesetzt sind.
- `PYANNOTE_HF_TOKEN` ist dein persönlicher Hugging Face Authentication Token, notwendig zum Laden des Pyannote Modells.
- `PYANNOTE_MODEL` ist die Pfadangabe des Modells, z. B. `"pyannote/speaker-diarization"`.


In [2]:
# Laden der Variablen aus .env (falls vorhanden)
load_dotenv()

PYANNOTE_HF_TOKEN = os.getenv("PYANNOTE_HF_TOKEN", "")
PYANNOTE_MODEL = os.getenv("PYANNOTE_MODEL", "PYANNOTE_MODEL")

### 3. Dateipfade spezifizieren
- Wir definieren Ordner und Dateinamen (z. B. `audio_file_folder`, `audio_name`, `audio_format`, `pyannote_folder`).
- Lege in `audio_file_folder` deine eigenen Audiodateien ab und ändere `audio_name` entsprechend, um deine eigenen Dateien zu transkribieren
- Damit wir die Ausgabe von Pyannote nachher weiterverwenden können, speichern wir die Ausgabe später in einer .json File im `pyannote_folder` ab. 

Wir benötigen eine **.wav** Datei als Eingabe für Pyannote. Mit **ffmpeg** können wir Audiodateien von anderem Format konvertieren.

In [3]:
!ffmpeg -i ./audios/<audio_name>.m4a ./audios/<audio_name>.wav

/bin/bash: line 1: audio_name: No such file or directory


In [3]:
# Spezifierung von Verzeichnissen und Dateinamen
audio_file_folder = "./audios/"
audio_name = "Sonic-Manus-Pitch"
audio_format = ".wav"
pyannote_folder = "./outputs/pyannote/"

### 4. Pipeline laden und Audio verarbeiten

- `Pipeline.from_pretrained(PYANNOTE_MODEL, use_auth_token=...)` lädt das gewählte Pyannote-Modell. 
- Anschließend rufen wir `pipeline(audio_file_path)` auf, um das Audio zu diarizieren.
- Das Ergebnis (`diarization`) enthält die zeitlichen Segmente und die vermuteten Sprecher (Labels).


In [4]:
# Laden der Diarization-Pipeline
pipeline = Pipeline.from_pretrained(
  PYANNOTE_MODEL,
  use_auth_token=PYANNOTE_HF_TOKEN
)

# Audio verarbeiten
diarization = pipeline(f"{audio_file_folder}{audio_name}{audio_format}")

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
/usr/local/lib/python3.12/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


: 

### 5. Iteration über die Diarization-Resultate

- `diarization.itertracks(yield_label=True)` liefert für jeden `turn` den Zeitraum (`start`, `end`) und das `speaker`-Label.
- Wir sammeln diese in einer Liste (`diarization_data`)

So erhält man am Ende ein Array mit Einträgen wie:
```json
{
  "start": 0.0,
  "end": 1.23,
  "speaker": "SPEAKER_00"
}
```

- Mit `json.dumps(diarization_data, indent=4)` können wir das gesamte Dictionary (mit Timestamps und Speaker Labels etc.) schön formatiert ausgeben.

In [ ]:
# Ergebnisse in strukturierter Form sammeln
diarization_data = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    diarization_data.append({
        "start": turn.start,
        "end": turn.end,
        "speaker": speaker
    })

# Ergebnis als JSON ausgeben
print(json.dumps(diarization_data, indent=4))


[
    {
        "start": 1.26284375,
        "end": 10.240343750000001,
        "speaker": "SPEAKER_00"
    },
    {
        "start": 10.797218750000003,
        "end": 12.14721875,
        "speaker": "SPEAKER_00"
    },
    {
        "start": 12.450968750000001,
        "end": 13.46346875,
        "speaker": "SPEAKER_01"
    },
    {
        "start": 13.547843750000002,
        "end": 20.078468750000003,
        "speaker": "SPEAKER_01"
    },
    {
        "start": 21.04034375,
        "end": 22.57596875,
        "speaker": "SPEAKER_00"
    },
    {
        "start": 22.91346875,
        "end": 27.18284375,
        "speaker": "SPEAKER_01"
    },
    {
        "start": 27.97596875,
        "end": 30.675968750000003,
        "speaker": "SPEAKER_00"
    },
    {
        "start": 31.13159375,
        "end": 32.11034375,
        "speaker": "SPEAKER_00"
    },
    {
        "start": 33.494093750000005,
        "end": 34.47284375,
        "speaker": "SPEAKER_00"
    },
    {
        "start": 

### 6. Datei Speichern
- `diarization_data` enthält das Ergebniss der Diarization (Sprechertrennung).
- Nun speichern wir das Ergebnis in einer `.json`-Datei.

In [ ]:
# In Datei speichern
with open(f"{pyannote_folder}{audio_name}.json", "w") as json_file:
    json.dump(diarization_data, json_file, indent=2)

## Zusammenfassung

Hier wurde gezeigt, wie man eine **Speaker Diarization** mithilfe von `pyannote.audio` durchführt:
1. Laden von Modell und Token aus den Umgebungsvariablen (`dotenv`).
2. Anlegen einer Pipeline (z. B. `pyannote/speaker-diarization`).
3. Verarbeitung der Audiodatei und Iteration über die Zeitschnitte.
4. Speichern der Ergebnisse als strukturierte JSON-Datei, um später darauf zugreifen zu können.

Nun kannst du die **Sprecherabschnitte** z. B. für Transkript-Annotationen, Audio-Schnitt, oder jede andere Anwendung nutzen, bei der du wissen musst, *wer wann spricht*.
